# Peace Treaties (one page)

The website that I'm scraping here is a database that contains metadata on European peace treaties from 1450 bis 1789. It was created as part of a project that was funded by the DFG (German Research Foundation): https://www.ieg-friedensvertraege.de/vertraege

The collection contains a selection of some 1800 treaties.

### Import libraries and packeges

In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import re
import csv

**re** is not required if you don't use regex to extract text. Python3 works better for this script; Python2 doesn't handle utf-8 too well.

This is added to slow down requests rate from the website. It is probably not necessary for such a small database, but it is good practice not to stress a website with automated requests.

In [2]:
from time import sleep
from random import randint

### Set up the containers for capturing treaty names, thes sides involved and the dates of the treaty:

In [3]:
treatytitles = []
treatysides = []
treatydates = []

### Indicate the url

You need to indicate the url of the webpage in order to scrape the data. Parser is html in this case.

In [4]:
url = "https://www.ieg-friedensvertraege.de/likecms.php?searchlang=de&step2onpage=3&function=process&process_target=ieg_treaty_mask_step2&site=index.html&nav=1&siteid=2&formlang=de&date=&year_from=&year_till=&location=&partner1=&partner2=&partner3=&language=&archive=&limit=&submit2=suche"
results = requests.get(url) 
soup = BeautifulSoup(results.text, "html.parser")

This alternative url is for filtered results only containing treaties involving Russia, using the websites search function. There are more elegant ways to do this, but this works.

In [5]:
# url="https://www.ieg-friedensvertraege.de/likecms.php?searchlang=de&step2onpage=3&function=process&process_target=ieg_treaty_mask_step2&site=index.html&nav=1&siteid=2&formlang=de&date=&year_from=&year_till=&location=&partner1=9&partner2=&partner3=&language=&archive=&limit=&submit2=suche"
# results = requests.get(url) 
# soup = BeautifulSoup(results.text, "html.parser")

### Write scraping code

If you don't know how to read the html text of a website to identify your data, you should check one of the tutorials mentioned in the repository description.

This is the scraping code for titles, dates, and sides. Sides was the most difficult to get, because it is not defined. It just sits between other bits of data. Apparently it always comes immediately after 'br' in td[1], so it can be grabbed with next_sibling, but that also grabs whitespace and newlines.

In [6]:
treaties = soup.find_all('tr', class_='text13')

sleep(randint(2,10))
    
for container in treaties:
    td = container.find_all('td')
    
    title = td[1].b.a.get_text() 
    treatytitles.append(title)
    
    sides = td[1].find('br').next_sibling #works but includes the whitespace and newline as well
    treatysides.append(sides)
    
    dates = td[2].get_text()
    treatydates.append(dates)

**treaties** pre-defines the frame within which I will look for the data. 
If you look at web page content using Web Developer Tools in the browser, you will find that all the database information (titles, sides, and dates) can be found within a series of **tr** (table row) tags. I have further specified the class ('text13'), to make sure I get the correct table rows in case the tag is used elsewhere on the page.

All subsequent searches will refer to this **treaties** container.

**sleep** is pausing requests from website. Random pause between 2-10 seconds.

Within each **tr** tag, there are three **td** (table data) tags. One for the number of the database entry (td[0]). One for the treaty title and the sides involved (td[1]), and one for the dates (td[2]). Note that counting them starts with 0!

By referring to the respective **td** tag by their number, I can then further indicate where the text that I want to grab is located within that tag.

The title data, for example, is located within an **a** tag that is again contained within a **b** tag. **get_text()** grabs just the text content within.

### Fill the containers with the scraped data:

In [7]:
out_treaties = pd.DataFrame({
 'title': treatytitles,
'sides': treatysides,
 'dates': treatydates
})

If you keep appending (because you are re-running the code, because something went wrong the first time), it piles up and has double, triple the rows, so delete and recreate containers too:

In [8]:
#del(out_treaties)

The following code gets rid of all newlines in the dataframe.

In [9]:
new_treaties = out_treaties.replace('\n',' ', regex=True)

Get a preview of the data

In [10]:
print(new_treaties)

                                                  title  \
0     Heiratsvertrag von Romans-sur-Isère zw. Charlo...   
1                           Friedensvertrag von Ferrara   
2     Leonello d'Este ordnet Gefangenenaustausch an ...   
3                             Friedensvertrag von Genua   
4                                       Friedensvertrag   
...                                                 ...   
1831                           Zweiter Waffenstillstand   
1832                           Dritter Waffenstillstand   
1833  Konvention von Hamburg zur Verlängerung des Ha...   
1834        Freundschafts- und Handelsvertrag von Genua   
1835  Garantieakte von Genf für die Verfassung der R...   

                                                  sides        dates  
0                         Frankreich, Savoyen - Piemont    1450 I 28  
1                        Aragon / Neapel - Sizilien ...   1450 VII 2  
2                        Aragon / Neapel - Sizilien ...   1450 VII 2  
3      

### Export dataframe as csv file

If you are happy with the results, you can export the dataframe to a CSV file.

In [11]:
new_treaties.to_csv('new_treaties_all.csv')

## Create Network Data

The following code restructures the data obtained from scraping in order to use it for network analysis. In this case, I will create two nodelists and one edgelist from the data in order to generate a bipartite graph with treaties and countries as nodes.

One of the challenges with the original data structure is the fact that the *sides* column contains multiple entries. First transform the structure within the sides column.

In [12]:
sides_lists = new_treaties.sides.str.split(',')         # split strings into list
new_treaties.sides = sides_lists                        # replace strings with lists in the dataframe

In [13]:
sides_lists

0       [                   Frankreich,  Savoyen - Pie...
1       [                   Aragon / Neapel - Sizilien...
2       [                   Aragon / Neapel - Sizilien...
3          [                   Genua,  Savoyen - Piemont]
4        [                   Mailand,  Savoyen - Piemont]
                              ...                        
1831             [                   Dänemark,  Schweden]
1832             [                   Dänemark,  Schweden]
1833            [                   Frankreich,  Hamburg]
1834                [                   Dänemark,  Genua]
1835    [                   Bern,  Frankreich,  Sardin...
Name: sides, Length: 1836, dtype: object

The explode method worked best to create a full list of the sides. It duplicates title and dates entries.

In [14]:
df3 = new_treaties.explode('sides') # explode based on the sides column

In [15]:
df3

,title,sides,dates
0,Heiratsvertrag von Romans-sur-Isère zw. Charlo...,Frankreich,1450 I 28
0,Heiratsvertrag von Romans-sur-Isère zw. Charlo...,Savoyen - Piemont,1450 I 28
1,Friedensvertrag von Ferrara,Aragon / Neapel - Sizilien,1450 VII 2
1,Friedensvertrag von Ferrara,Ferrara – Modena – (Reggio),1450 VII 2
1,Friedensvertrag von Ferrara,Venedig,1450 VII 2
...,...,...,...
1834,Freundschafts- und Handelsvertrag von Genua,Dänemark,1789 VI 30
1834,Freundschafts- und Handelsvertrag von Genua,Genua,1789 VI 30
1835,Garantieakte von Genf für die Verfassung der R...,Bern,1789 XII 9
1835,Garantieakte von Genf für die Verfassung der R...,Frankreich,1789 XII 9


Remove whitespace around strings in sides. Avoids false duplicates.

In [16]:
df3.sides = df3.sides.str.strip()

This dataframe will later serve as the basis for the edgelist, but first I will extract the nodelists and add Ids.

### Extract unique nodes (countries)

In [17]:
SideNodes = df3.sides.drop_duplicates().reset_index()

Sort them alphabetically:

In [18]:
SideNodes.sort_values('sides')

,index,sides
101,373,Anhalt - Zerbst
178,1756,Ansbach
24,34,Appenzell
139,705,Appenzell Außerrhoden
104,412,Appenzell Innerrhoden
...,...,...
176,1724,Würzburg (Hochstift)
68,174,Zehngerichtenbund
19,13,Zug
20,13,Zürich


Add Ids to nodelists and edgelists. Because I want to create a bipartite graph, I am adding alphanumnerical Ids.

In [19]:
SideNodes['Side_Id'] = SideNodes.index +1

In [20]:
SideNodes['IDLetter'] = 'C'

Concatenate the 2 columns, convert numbers to strings.

In [21]:
SideNodes['Side_ID'] = SideNodes["IDLetter"] + SideNodes["Side_Id"].astype(str)

In [22]:
SideNodes

,index,sides,Side_Id,IDLetter,Side_ID
0,0,Frankreich,1,C,C1
1,0,Savoyen - Piemont,2,C,C2
2,1,Aragon / Neapel - Sizilien,3,C,C3
3,1,Ferrara – Modena – (Reggio),4,C,C4
4,1,Venedig,5,C,C5
...,...,...,...,...,...
176,1724,Würzburg (Hochstift),177,C,C177
177,1747,Ragusa,178,C,C178
178,1756,Ansbach,179,C,C179
179,1758,Sachsen - Coburg,180,C,C180


Select only those columns that you need for the nodelist:

In [24]:
Country_nodes = SideNodes.loc[:, ["Side_ID", "sides", 'Side_Id']]

In [28]:
# Country_nodes #preview of country nodes

Export country nodes to a CSV file and drop the original index from the dataframe:

In [26]:
Country_nodes.to_csv('Country_nodes_all.csv', index= False)

### Extract unique nodes (treaties)

In [29]:
TreatyNodes = df3.title.drop_duplicates().reset_index()

In [30]:
TreatyNodes['Treaty_Id'] = TreatyNodes.index +1

In [31]:
TreatyNodes['Letter'] = 'T'

In [32]:
TreatyNodes['Treaty_ID'] = TreatyNodes["Letter"] + TreatyNodes["Treaty_Id"].astype(str)

In [33]:
TreatyNodes

,index,title,Treaty_Id,Letter,Treaty_ID
0,0,Heiratsvertrag von Romans-sur-Isère zw. Charlo...,1,T,T1
1,1,Friedensvertrag von Ferrara,2,T,T2
2,2,Leonello d'Este ordnet Gefangenenaustausch an ...,3,T,T3
3,3,Friedensvertrag von Genua,4,T,T4
4,4,Friedensvertrag,5,T,T5
...,...,...,...,...,...
1457,1831,Zweiter Waffenstillstand,1458,T,T1458
1458,1832,Dritter Waffenstillstand,1459,T,T1459
1459,1833,Konvention von Hamburg zur Verlängerung des Ha...,1460,T,T1460
1460,1834,Freundschafts- und Handelsvertrag von Genua,1461,T,T1461


In [34]:
Treaty_nodes = TreatyNodes.loc[:, ["Treaty_ID", "title", 'Treaty_Id']]

In [35]:
Treaty_nodes

,Treaty_ID,title,Treaty_Id
0,T1,Heiratsvertrag von Romans-sur-Isère zw. Charlo...,1
1,T2,Friedensvertrag von Ferrara,2
2,T3,Leonello d'Este ordnet Gefangenenaustausch an ...,3
3,T4,Friedensvertrag von Genua,4
4,T5,Friedensvertrag,5
...,...,...,...
1457,T1458,Zweiter Waffenstillstand,1458
1458,T1459,Dritter Waffenstillstand,1459
1459,T1460,Konvention von Hamburg zur Verlängerung des Ha...,1460
1460,T1461,Freundschafts- und Handelsvertrag von Genua,1461


Export treaty nodes to CSV

In [36]:
Treaty_nodes.to_csv('Treaty_nodes_all.csv', index= False)

### Create the edgelist
Now add Ids to edgelist with left join from both nodelists

In [37]:
# Merge DataFrames by Column
df4=pd.merge(TreatyNodes,df3, on='title', how='left')

In [38]:
df5=pd.merge(SideNodes,df4, on='sides', how='left')

In [39]:
df5

,index_x,sides,Side_Id,IDLetter,Side_ID,index_y,title,Treaty_Id,Letter,Treaty_ID,dates
0,0,Frankreich,1,C,C1,0,Heiratsvertrag von Romans-sur-Isère zw. Charlo...,1,T,T1,1450 I 28
1,0,Frankreich,1,C,C1,3,Friedensvertrag von Genua,4,T,T4,1466 VIII 21
2,0,Frankreich,1,C,C1,5,Bündnisvertrag von Chambéry,6,T,T6,1451 III 13
3,0,Frankreich,1,C,C1,13,"Bündnis- und Freundschaftsvertrag, Eidgen. Erk...",14,T,T14,1452 XI 08
4,0,Frankreich,1,C,C1,15,"Bündnis- und Freundschaftsvertrag, kgl. Erklär...",16,T,T16,1453 II 27
...,...,...,...,...,...,...,...,...,...,...,...
5307,1724,Würzburg (Hochstift),177,C,C177,1725,Vertrag von Versailles (betr. Droit d' Aubaine),1390,T,T1390,1773 VI 1
5308,1747,Ragusa,178,C,C178,1747,Konvention von Ragusa zu Handel und Schiffahrt,1404,T,T1404,1776 IV 2
5309,1756,Ansbach,179,C,C179,1701,Versailler Aufhebungsvertrag über das Droit d'...,1371,T,T1371,1778 II 19
5310,1758,Sachsen - Coburg,180,C,C180,1701,Versailler Aufhebungsvertrag über das Droit d'...,1371,T,T1371,1778 IV 7


Select relevant columns for edgelist.

In [40]:
Treaty_edges = df5.loc[:, ["Treaty_ID", "Side_ID", 'dates']]

In [41]:
Treaty_edges

,Treaty_ID,Side_ID,dates
0,T1,C1,1450 I 28
1,T4,C1,1466 VIII 21
2,T6,C1,1451 III 13
3,T14,C1,1452 XI 08
4,T16,C1,1453 II 27
...,...,...,...
5307,T1390,C177,1773 VI 1
5308,T1404,C178,1776 IV 2
5309,T1371,C179,1778 II 19
5310,T1371,C180,1778 IV 7


## Deal with dates
The dates are in an inconvenient format that is difficult to process. The month is represented in Roman numerals, which in itself is not a problem (they can be converted into a regular date format), but some dates are given as a time-range, rather than one single date (for example: 1663 VII 24_VIII 3). In this example, I will just export the year to a new column to be able to ise it later. 

In [52]:
# regex definition get year from the data
def get_year(file):
  year_match = re.findall(r'(\d{4})',file)
  return year_match[0]

In [53]:
# # regex definition get year from the data
# def get_year(file):
#   year_match = re.findall(r'_(\d{4})-\d{2}-\d{2}_',file)
#   return year_match[0]

Add year data to a new column

In [54]:
Treaty_edges['year'] = Treaty_edges['dates'].apply(lambda x: get_year(x))

In [58]:
Treaty_edges.sort_values('year') #check the new dataframe and sort by year column

,Treaty_ID,Side_ID,dates,year
0,T1,C1,1450 I 28,1450
910,T2,C4,1450 VII 2,1450
911,T3,C4,1450 VII 2,1450
916,T2,C5,1450 VII 2,1450
917,T3,C5,1450 VII 2,1450
...,...,...,...,...
1042,T1461,C6,1789 VI 30,1789
4967,T1460,C138,1789 III 17,1789
5282,T1462,C165,1789 XII 9,1789
1222,T1462,C9,1789 XII 9,1789


Export final edgelist to CSV

In [59]:
Treaty_edges.to_csv('Treaty_edges_all.csv', index= False)